In [21]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import re
from curses.ascii import isalpha, isdigit

In [2]:
df = pd.read_csv('final_kun_uz_dataset.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172349 entries, 0 to 172348
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   title    172349 non-null  object
 1   content  172349 non-null  object
 2   tag      172349 non-null  object
dtypes: object(3)
memory usage: 5.3+ MB


In [4]:
df 

,title,content,tag
0,"Eng kuchli Android-smartfonlar, uchar mehmonxo...",Eng kuchli Android-smartfonlarAvgust oyining e...,fan va texnika
1,"Apple taqdimoti: Yangi qurilmalar, o‘zgarishla...",7 sentabr kuni Apple kompaniyasi iPhone 14 lin...,fan va texnika
2,Eng kuchli Android-smartfonlar reytingi e’lon ...,Birinchi o‘rinni Asus ROG Phone 6 smartfoni eg...,fan va texnika
3,Sun’iy intellekt va xavfsizlik. Robotlar odaml...,Globallashgan dunyoda rivojlanishning asosiy o...,fan va texnika
4,Apple iPhone 14 va iPhone 14 Plus smartfonlari...,Apple kompaniyasi iPhone 14 va iPhone 14 Plus ...,fan va texnika
...,...,...,...
172344,Ispaniya klubini bir yo‘la barcha futbolchilar...,Ispaniya Segundasida ishtirok etuvchi «Reus» k...,sport
172345,«Inter» «Oltin to‘p» sohibini o‘z safiga qo‘sh...,«Inter» bosh direktori Juzeppe Marotta jamoani...,sport
172346,To‘lqin Qilichev: «Yosh bokschini shogird qili...,Boks bo‘yicha O‘zbekiston terma jamoasi murabb...,sport
172347,"Angliyada «qizillar» bahsi, Ispaniyada kichik ...",Dam olish kunlarida Yevropa chempionatlarida n...,sport


In [10]:
list2 = []
list3 = []
for i in range(len(df['content'])):
    str1 = df['content'][i]
    list1 = str1.split()
     
    try:
        df['content'][i] = " ".join(text_ichida_(list1)) 
    except ValueError:
        list2.append(i)
    except IndexError:
        list3.append(i)

In [11]:
list2

[8742, 29438, 67203, 87787]

In [20]:
df.drop(87787, axis=0, inplace=True)

In [17]:
df.loc[8744]

title      Google dasturlarni yuklashga undovchi soxta tu...
content    Google kompaniyasi foydalanuvchilarni fishing ...
tag                                           fan va texnika
Name: 8744, dtype: object

In [22]:
def kichik_harf_(df):
    df = df.apply(lambda x: x.astype(str).str.lower())
    return df
kichik_harf = FunctionTransformer(kichik_harf_)


In [23]:
def apastrof_(df):
    df['title']=df['title'].map(lambda x: re.sub(r"'|`|´|‘",r"’", x))
    df['content']=df['content'].map(lambda x: re.sub(r"'|`|´|‘",r"’", x))
    df['title']=df['title'].map(lambda x: re.sub(r"(o|g)’",r"\1‘",x))
    df['content']=df['content'].map(lambda x: re.sub(r"(o|g)’",r"\1‘",x))
    df['tag']=df['tag'].map(lambda x: re.sub(r"(o)'",r"\1‘",x))
    return df
apastrof = FunctionTransformer(apastrof_)

In [24]:
def numToWords(num,join=True):
    num = int(float(num))
    units = ["","bir","ikki",'uch',"to‘rt",'besh','olti','yetti','sakkiz',"to‘qqiz"]
    tens = ['',"o‘n",'yigirma',"o‘ttiz",'qirq','ellik','oltmish','yetmish', 'sakson',"to‘qson"]
    thousands = ['','ming','million','milliard','trillion','kvadrillion','kvintilion','sekstilion','septillion','oktilion', 'nonillion','dekillion','undekillion','duodekillion', 'tredekillion','quattuordekillion','sexdekillion', 'septendekillion','oktodekillion','novemdekillion', 'vigintillion', 'unvigintillion','duovigintillion', 'trevigintillion', 'quattourvigintillion' , 'quinvigintillion', 'hexvigintillion', 'septenvigintillion', 'octovigintillion', 'novemvigintillion', 'trigintillion', 'untrigintillion', 'duotrigintillion', 'googol']
    words = []
    if num==0: words.append('nol')
    else:
        numStr = '%d'%num
        numStrLen = len(numStr)
        
        groups = (numStrLen+2)//3
        numStr = numStr.zfill(groups*3)
        for i in range(0,groups*3,3):
            h,t,u = int(numStr[i]),int(numStr[i+1]),int(numStr[i+2])
            g = groups-(i//3+1)
            if h>=1:
                words.append(units[h])
                words.append('yuz')
            if t>1:
                words.append(tens[t])
                if u>=1: words.append(units[u])
            elif t==1:
                if u>=1: words.append( tens[1]+" " + units[u])
                else: words.append(tens[t])
            else:
                if u>=1: words.append(units[u])
            if (g>=1) and ((h+t+u)>0): words.append(thousands[g]+' ')
    if join: return ' '.join(words)
    return words

def is_number(s):
    try:
        if '.' in s:
            return float(s)
        elif ',' in s:
            return  float(s.split(',')[0] + '.'+ s.split(',')[1])
        else:
            return int(s)
    except ValueError:
        return s
    
def text_ichida_(str_text):
    for i in range(len(str_text)):
        if isinstance(is_number(str_text[i]), float):
            if is_number(str_text[i]) > 0:
                k1 = str(is_number(str_text[i])).split('.')
                str_text[i] = f"{numToWords(int(k1[0]))} butun {numToWords(int(k1[1]))}"
            elif is_number(str_text[i]) < 0:
                k2 = str(is_number(str_text[i]))[1:].split('.')
                str_text[i] = f" minus {numToWords(int(k2[0]))} butun {numToWords(int(k2[1]))}"
            elif is_number(str_text[i]) == 0.0:
                k3 = str(is_number(str_text[i])).split('.')
                str_text[i] = f"{numToWords(int(k3[0]))} butun {numToWords(int(k3[1]))}"
        elif isinstance(is_number(str_text[i]), int):
            if is_number(str_text[i]) > 0:
                str_text[i] = numToWords(is_number(str_text[i]))
            else: 
                str_text[i] = f" minus {numToWords(abs(is_number(str_text[i])))}"
    return str_text


In [25]:
def raqamchi_(df):
    df['title'] = df['title'].map(lambda x: " ".join(text_ichida_(x.split())))
    df['content'] = df['content'].map(lambda x: " ".join(text_ichida_(x.split())))
    return df
raqamchi = FunctionTransformer(raqamchi_)

In [26]:
def regex_for_numbers_(df):
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+)-([0-9]+)(\w*)', r'\1 \2 \3', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+)-([0-9]+)(\w*)', r'\1 \2 \3', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'(\d+)\.([a-z]+)', r'\1 . \2', x))
    df['content']=df['content'].map(lambda x: re.sub(r'(\d+)\.([a-z]+)', r'\1 . \2', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+)-([a-z]+)(\d+)', r'\1 - \2 \3 ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+)-([a-z]+)(\d+)', r'\1 - \2 \3 ', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+)-(\d{1,9})-([a-z]+)', r'\1 - \2 - \3', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+)-(\d{1,9})-([a-z]+)', r'\1 - \2 - \3', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'(\d)-(\d)-(\d)-(\d)', r'\1 - \2 - \3 - \4', x))
    df['content']=df['content'].map(lambda x: re.sub(r'(\d)-(\d)-(\d)-(\d)', r'\1 - \2 - \3 - \4', x)) 

    df['title']=df['title'].map(lambda x: re.sub(r'(\D)(\d{4})(\d{4})(\D)', r"\1 \2 - \3 \4 yil oralig'ida ", x))
    df['content']=df['content'].map(lambda x: re.sub(r'(\D)(\d{4})(\d{4})(\D)', r"\1 \2 - \3 \4 yil oralig'ida ", x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'(\")([0-9]+)', r'\1 \2 ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'(\")([0-9]+)', r'\1 \2 ', x)) 

    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)', r'\1 - \2 - \3 ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)', r'\1 - \2 - \3 ', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9])+-([0-9]+)-([a-z]+)', r'\1 inchi \2 inchi \3', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9])+-([0-9]+)-([a-z]+)', r'\1 inchi \2 inchi \3', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+\.)([0-9]+)-([0-9]+)(\.)([a-z]+)', r'\1 \2 - \3 \4 \5', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+\.)([0-9]+)-([0-9]+)(\.)([a-z]+)', r'\1 \2 - \3 \4 \5', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)-([0-9]+)([a-z]+)', r'\1 - \2 \3', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)-([0-9]+)([a-z]+)', r'\1 - \2 \3', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)-([0-9]+)', r'\1 inchi \2 inchi', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)-([0-9]+)', r'\1 inchi \2 inchi', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+\.)([0-9]+)(\.)', r'\1 \2 \3 ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+\.)([0-9]+)(\.)', r'\1 \2 \3 ', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+\.)([0-9]+)', r'\1 \2 ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+\.)([0-9]+)', r'\1 \2 ', x)) 
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+)([0-9]+)([a-z]+)', r' ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+)([0-9]+)([a-z]+)', r' ', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)-([a-z]+)', r'\1 inchi \2', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)-([a-z]+)', r'\1 inchi \2', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'([a-z]+)([0-9]+)', r'\1 \2', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([a-z]+)([0-9]+)', r'\1 \2', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)([a-z]+)', r'\1 \2', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)([a-z]+)', r'\1 \2', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)(\’[a-z]+)', r'\1 \2', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)(\’[a-z]+)', r'\1 \2', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'([0-9]+)(\-)', r'\1 inchi \2', x))
    df['content']=df['content'].map(lambda x: re.sub(r'([0-9]+)(\-)', r'\1 inchi \2', x))
    
    df['title']=df['title'].map(lambda x: re.sub(r'[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+', r' ', x))
    df['content']=df['content'].map(lambda x: re.sub(r'[^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+', r' ', x)) 
    df['title']=df['title'].map(lambda x: re.sub(r':', r' ', x))
    df['content']=df['content'].map(lambda x: re.sub(r':', r' ', x)) 
    return df
regex_for_numbers = FunctionTransformer(regex_for_numbers_)

In [27]:
def inchi_(df):
    df['title'] = df['title'].map(lambda x: re.sub(r"  inchi",r'inchi' , x))
    df['content'] = df['content'].map(lambda x: re.sub(r"  inchi",r'inchi' , x))
    df['title'] = df['title'].map(lambda x: re.sub(r" inchi",r'inchi' , x))
    df['content'] = df['content'].map(lambda x: re.sub(r" inchi",r'inchi' , x))
    df['title'] = df['title'].map(lambda x: re.sub(r"ii",r'i' , x))
    df['content'] = df['content'].map(lambda x: re.sub(r"ii",r'i' , x))
    return df
inchi = FunctionTransformer(inchi_)

In [28]:
def keraksiz_belgilar_(df):
    df['title'] = df['title'].map(lambda x: re.sub(r"[^0-9a-z‘’]+",r" ", x))
    df['content'] = df['content'].map(lambda x: re.sub(r"[^0-9a-z‘’]+",r" ", x))
    return df
keraksiz_belgilar = FunctionTransformer(keraksiz_belgilar_)

In [29]:
full_pipeline = Pipeline(
[
    ('kichik_harf', kichik_harf),
    ('apastrof', apastrof),
    ('regeex', regex_for_numbers),
    ('raqamchi', raqamchi),
    ('inchi',inchi),
    ('keraksiz_belgilar',keraksiz_belgilar)
])

In [30]:
df2 = full_pipeline.transform(df)

In [61]:
df2

,title,content,tag
0,eng kuchli android smartfonlar uchar mehmonxon...,eng kuchli android smartfonlaravgust oyining e...,fan va texnika
1,apple taqdimoti yangi qurilmalar o‘zgarishlar ...,7 sentabr kuni apple kompaniyasi iphone 14 lin...,fan va texnika
2,eng kuchli android smartfonlar reytingi e’lon ...,birinchi o‘rinni asus rog phone 6 smartfoni eg...,fan va texnika
3,sun’iy intellekt va xavfsizlik robotlar odamla...,globallashgan dunyoda rivojlanishning asosiy o...,fan va texnika
4,apple iphone 14 va iphone 14 plus smartfonlari...,apple kompaniyasi iphone 14 va iphone 14 plus ...,fan va texnika
...,...,...,...
172344,ispaniya klubini bir yo‘la barcha futbolchilar...,ispaniya segundasida ishtirok etuvchi reus klu...,sport
172345,inter oltin to‘p sohibini o‘z safiga qo‘shib ...,inter bosh direktori juzeppe marotta jamoanin...,sport
172346,to‘lqin qilichev yosh bokschini shogird qilib ...,boks bo‘yicha o‘zbekiston terma jamoasi murabb...,sport
172347,angliyada qizillar bahsi ispaniyada kichik mad...,dam olish kunlarida yevropa chempionatlarida n...,sport


In [31]:
df2 

,title,content,tag
0,eng kuchli android smartfonlar uchar mehmonxon...,eng kuchli android smartfonlaravgust oyining e...,fan va texnika
1,apple taqdimoti yangi qurilmalar o‘zgarishlar ...,yetti sentabr kuni apple kompaniyasi iphone o‘...,fan va texnika
2,eng kuchli android smartfonlar reytingi e’lon ...,birinchi o‘rinni asus rog phone olti smartfoni...,fan va texnika
3,sun’iy intellekt va xavfsizlik robotlar odamla...,globallashgan dunyoda rivojlanishning asosiy o...,fan va texnika
4,apple iphone o‘n to‘rt va iphone o‘n to‘rt plu...,apple kompaniyasi iphone o‘n to‘rt va iphone o...,fan va texnika
...,...,...,...
172344,ispaniya klubini bir yo‘la barcha futbolchilar...,ispaniya segundasida ishtirok etuvchi reus klu...,sport
172345,inter oltin to‘p sohibini o‘z safiga qo‘shib ...,inter bosh direktori juzeppe marotta jamoanin...,sport
172346,to‘lqin qilichev yosh bokschini shogird qilib ...,boks bo‘yicha o‘zbekiston terma jamoasi murabb...,sport
172347,angliyada qizillar bahsi ispaniyada kichik mad...,dam olish kunlarida yevropa chempionatlarida n...,sport


In [38]:
df2['content'][170001]

'tunisda para atletika bo‘yicha gran pri seriyasiga kiruvchi yirik musobaqa o‘z yakuniga yetdi ellik ga yaqin mamlakatdan tashrif buyurgan para atletlar ishtirokidagi mazkur musobaqa o‘zbekiston terma jamoasi uchun omadli yakunlandi turli yo‘nalishlarda startga chiqqan o‘zbekistonlik sportchilar gran prida olti ta oltin uch ta kumush va bir ta bronza jami o‘n ta medal qo‘lga kiritishga muvaffaq bo‘lishdi ushbu musobaqa orqali o‘zbekiston vakillari jahon chempionatiga ham yo‘llanmaga ega bo‘lishdi noyabr oyida baada bo‘lib o‘tadigan dunyo birinchiligi esa kelasi yili tokioda o‘tkaziladigan paralimpiya o‘yinlariga yo‘llanma taqdim etadi musobaqada g‘olib va sovrindor bo‘lgan o‘zbekistonlik sportchilar ro‘yxati oltin doniyor axmedov bir yuz va ikki yuz metrga yugurish mohigul hamdamova disk uloqtirish omad hasanov balandlikka sakrash bekjon chevarov uzunlikka sakrash mohigul hamdamova nayza otishkumush ma’ruf murodullayev bir ming besh yuz metrga yugurish mohigul hamdamova yadro uloqtiris

In [36]:
from collections import Counter

Counter(''.join(df2.unstack().values))  

Counter({'e': 7398689,
         'n': 19277630,
         'g': 9684987,
         ' ': 43298118,
         'k': 9080363,
         'u': 9437879,
         'c': 3287095,
         'h': 12525672,
         'l': 17828484,
         'i': 40811736,
         'a': 43879419,
         'd': 10839751,
         'r': 17208323,
         'o': 20072629,
         's': 14967923,
         'm': 10036002,
         't': 14566118,
         'f': 1678395,
         'x': 1669462,
         'v': 4444821,
         'b': 9079291,
         'y': 8723414,
         'j': 1904761,
         'p': 1809602,
         'q': 6273614,
         '‘': 5528735,
         'z': 4546615,
         '’': 731131,
         'w': 48396,
         '8': 26228,
         '6': 24912,
         '2': 64424,
         '1': 66554,
         '5': 28098,
         '0': 70486,
         '9': 28262,
         '3': 26536,
         '4': 23787,
         '7': 26082})